In [5]:
 pip install opencv-contrib-python --upgrade

     |████████████████████████████████| 54.5 MB 647 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense # review


In [2]:
cnn = Sequential([Conv2D(filters=100, kernel_size = (3,3),
                    activation = 'relu'),
                    MaxPooling2D(pool_size = (2,2)),
                    Conv2D(filters = 100, kernel_size = (3,3),
                    activation = 'relu'),
                    MaxPooling2D(pool_size = (2,2)),
                    Flatten(),
                    Dropout(0.5),
                    Dense(50),
                    Dense(35),
                    Dense(2) ])

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [3]:
import cv2
import numpy as np


In [4]:

labels_dict = {0: 'No mask', 1: 'Mask'}
color_dict = {0: (0,0,255), 1: (0,255,0)}
imgsize = 4
camera = cv2.VideoCapture(0) # turn on the camera

#identify the front face 

classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


while True:
    (rval, im) = camera.read()
    im = cv2.flip(im,1,1) #mirror the image?
    imgs = cv2.resize(im, (im.shape[1] // imgsize, im.shape[0] // imgsize))
    face_rec = classifier.detectMultiScale(imgs)
    
    for i in face_rec: #overlay rectangle on the face 
        (x, y, l, w) = [v * imgsize for v in i ]
        face_img = im[y:y+w, x:x+l] # review needed
        resized = cv2.resize(face_img, (150,150))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1, 150, 150, 3))
        reshaped = np.vstack([reshaped])
        result = cnn.predict(reshaped)
        label = np.argmax(result, axis=1)[0]#review needed
        cv2.rectangle(im, (x,y), (x+l, y+w), color_dict[label],2)
        cv2.rectangle(im, (x,y-40), (x+l,y), color_dict[label], -1)
        cv2.putText(im, labels_dict[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2)
        
    cv2.imshow('LIVE', im)
    key = cv2.waitKey(10)
   
    #stop loop by ESC
    if key == 27: #ESC
        break
        
#webcam.release()
cv2.destroyAllWindows()
        
        
    


2021-11-10 12:44:44.614642: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-10 12:44:44.615128: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.
